In [1]:
import gzip
import json
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from Load import json_to_df
import preprocess
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import f1_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\caspe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Dataset

In [2]:
df = json_to_df("data\\music_reviews_train.json")
df

,verified,reviewTime,reviewerID,asin,reviewText,summary,unixReviewTime,sentiment,id,style.Format:,vote,image
0,True,"09 11, 2013",AC2PL52NKPL29,0001388703,"So creative! Love his music - the words, the ...",Gotta listen to this!,1378857600,positive,0,Audio CD,NaN,NaN
1,False,"05 14, 2009",A3NVGWKHLULDHR,0001388703,This tape can hardly be understood and it was ...,Shame Shame,1242259200,negative,1,Audio Cassette,NaN,NaN
2,True,"05 15, 2015",AZ3T21W6CW0MW,0001388703,Buy the CD. Do not buy the MP3 album. Downlo...,Buy the CD. Do not buy the MP3.,1431648000,negative,2,MP3 Music,NaN,NaN
3,True,"10 3, 2016",A50DSLM71EAVO,0001526146,I love Dallas Holms music and voice! Thank Yo...,Five Stars,1475452800,positive,3,NaN,NaN,NaN
4,False,"12 11, 2014",AKNKAT9XGAFPE,0001527134,Great memories of my early years in Christ,Five Stars,1418256000,positive,4,Audio CD,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,True,"10 7, 2015",A2M2XF2QJZS7T3,B002POOJN8,Great old song. Love it.,Great,1444176000,positive,99995,MP3 Music,NaN,NaN
99996,False,"12 17, 2012",A1BKW2O4G0DB9A,B002PQXDW4,Daniel Smith has no more command of the bassoo...,Absolute garbage,1355702400,negative,99996,Audio CD,3,NaN
99997,True,"11 4, 2017",A262BLYWUG3729,B002PRF7AO,I'm enjoying her album very much!,Five Stars,1509753600,positive,99997,NaN,NaN,NaN
99998,False,"06 24, 2017",A2O9U4W4O7KQH7,B002PRCQTE,These digital files are corrupted. There are ...,BAD DIGITAL RECORDING - Very sad,1498262400,negative,99998,MP3 Music,NaN,NaN


## Preproccesing
- using only the summary
- making the targets (0,1)
- dropping empty rows

### Train set

In [3]:
data = df[["summary","sentiment"]]

In [4]:
data = data.replace({"sentiment":{"positive":1,"negative":0}})

In [5]:
data = data.dropna()

### Validation set

In [8]:
df_dev = json_to_df("data\\music_reviews_dev.json")

In [9]:
data_dev = df_dev[["summary","sentiment"]]
data_dev = data_dev.replace({"sentiment":{"positive":1,"negative":0}})
data_dev = data_dev.dropna()

## Pipeline
- words counts
- naive bayes

In [6]:
pipeline = Pipeline(steps=[
    ("countervectorizer_transform",CountVectorizer()),
    ("MultinomialNB", MultinomialNB())
])

In [7]:
pipeline.fit(data["summary"],data["sentiment"])

Pipeline(steps=[('Tfidf_transform', CountVectorizer()),
                ('MultinomialNB', MultinomialNB())])

## Validation
Using f1 score to validate

In [10]:
preds = pipeline.predict(data_dev["summary"])

In [11]:
f1_score(preds, data_dev["sentiment"])

0.8980172490441896